In [73]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.linear_model import BayesianRidge
import re
import os
import warnings
warnings.filterwarnings("ignore")
s_dir = "./submission_0/"
pd.set_option('display.max_columns', None)
test = pd.read_csv("./datadown/test.csv")
y = pd.read_csv("./dataup/y.csv", )
y_origin = pd.read_csv("./datadown/train.csv", )
nomodel_sub = pd.read_csv("./submission_0/sub_nomodel.csv")

In [62]:

def scan_and_aggregate_submissions(base_dir):
    # Initialize empty DataFrames for each type
    oof_dfs = {}
    test_dfs = {}

    # Iterate over all directories in the base directory
    for dir_name in os.listdir(base_dir):
        dir_path = os.path.join(base_dir, dir_name)
        
        # Check if it's a directory
        if os.path.isdir(dir_path):
            # Initialize DataFrames for the current directory
            oof_df = pd.DataFrame()
            test_df = pd.DataFrame()

            # Iterate over all files in the directory
            for file_name in os.listdir(dir_path):
                file_path = os.path.join(dir_path, file_name)

                # Check if the file is a CSV and matches the pattern
                if file_name.endswith('.csv'):
                    if re.findall(r"_oof", file_name):
                        oof_df = pd.concat([oof_df, pd.read_csv(file_path)], axis=1)
                    elif re.findall(r"_test", file_name):
                        test_df = pd.concat([test_df, pd.read_csv(file_path)], axis=1)

            # Add a suffix to the columns to indicate the batch
            oof_df.columns = [f"{col}_{dir_name}" for col in oof_df.columns]
            test_df.columns = [f"{col}_{dir_name}" for col in test_df.columns]

            # Store the DataFrames in the dictionaries
            oof_dfs[dir_name] = oof_df
            test_dfs[dir_name] = test_df

    # Aggregate all DataFrames
    sm_oof_agg = pd.concat(oof_dfs.values(), axis=1)
    sm_test_agg = pd.concat(test_dfs.values(), axis=1)

    return sm_oof_agg, sm_test_agg

sm_oof_agg, sm_test_agg = scan_and_aggregate_submissions(s_dir)


In [71]:
oof_all = pd.concat([y_origin[['num_sold']].rename(columns={'num_sold': 'num_sold_origin'}), np.expm1(y), np.expm1(sm_oof_agg)], axis = 1)

In [75]:
oof_all['num_sold_origin'].fillna(oof_all['num_sold'], inplace=True)

In [110]:
nomodel_suboof_all

,id,num_sold
0,230130,158.0
1,230131,1031.0
2,230132,775.0
3,230133,458.0
4,230134,523.0
...,...,...
98545,328675,475.0
98546,328676,2924.0
98547,328677,2598.0
98548,328678,1331.0


In [65]:
test_all = np.expm1(sm_test_agg)

In [77]:
test_all

,CAT3_submissions_1,CAT_submissions_1,LGBM2_submissions_1,LGBM3_submissions_1,LGBM4_submissions_1,LGBM5_submissions_1,LGBM6_submissions_1,LGBM7_submissions_1,XGB2_submissions_1,XGB3_submissions_1,XGB4_submissions_1,XGB5_submissions_1,CAT3_submissions_2,CAT5_submissions_2,CAT6_submissions_2,CAT_submissions_2,LGBM2_submissions_2,LGBM3_submissions_2,LGBM4_submissions_2,LGBM5_submissions_2,LGBM6_submissions_2,LGBM7_submissions_2,NN_submissions_2,XGB2_submissions_2,XGB3_submissions_2,XGB4_submissions_2,XGB5_submissions_2,LGBM2_submissions_3,LGBM3_submissions_3,LGBM4_submissions_3,LGBM5_submissions_3,LGBM6_submissions_3,LGBM7_submissions_3,NN_submissions_3,BRidge_submissions_4,LGBM2_submissions_4,LGBM3_submissions_4,LGBM4_submissions_4,LGBM5_submissions_4,LGBM6_submissions_4,LGBM7_submissions_4,LR_submissions_4,NN_submissions_4,Ridge_submissions_4
0,133.305563,134.101903,130.519931,133.338966,130.892127,130.704232,133.405480,133.389273,137.914926,137.665007,137.572631,138.396404,139.868408,144.215874,144.484092,137.497733,132.351017,133.179533,132.897285,132.197548,133.841680,135.403559,123.351173,141.670717,142.357570,143.092002,142.587370,140.610265,140.012699,140.735028,139.673299,140.212269,139.727721,13.545432,147.124282,136.972497,137.787473,137.161501,138.064574,139.502293,137.623861,147.328224,167.969361,139.700730
1,843.624685,875.326324,812.718929,834.556594,816.769181,821.951422,836.304196,833.089710,915.619549,912.153962,907.761969,905.786472,863.897294,907.452281,911.658904,887.651873,828.482973,847.004219,833.205164,840.314147,852.634359,850.084299,802.902827,939.088481,937.849792,936.126239,928.422383,586.594228,620.430469,577.836791,591.375823,647.231056,575.295835,192.499901,1027.481374,891.201246,899.865476,903.144551,899.913596,913.496493,897.368912,1028.898211,880.113633,975.748814
2,692.197991,708.938729,699.694599,707.837513,702.086399,698.851979,704.471831,704.075644,771.253914,770.362167,773.499024,773.057840,715.841896,734.921337,743.056827,718.012170,710.989737,714.738196,715.661358,712.914869,712.479888,708.585353,648.824288,793.888617,791.719945,799.578027,795.380909,470.382979,492.814009,462.367121,471.483866,505.417116,463.107680,93.940720,687.013360,679.909390,686.418676,685.900076,685.328789,690.637293,686.070390,687.961012,594.736050,652.443244
3,369.113298,366.106250,377.054776,384.268327,378.310425,369.456390,386.534798,377.766777,415.136945,414.906843,413.806439,414.919611,383.179628,407.948247,407.942838,371.689634,387.041537,388.629749,387.333918,378.610657,388.791283,384.888090,350.828908,423.612505,424.846350,427.293925,425.937657,314.126503,325.514336,310.510704,316.981554,332.376250,308.751561,35.944223,405.262449,398.463110,404.543491,398.431985,401.537085,408.044829,401.643534,405.821912,447.436146,384.875908
4,474.079983,482.910927,451.721168,465.544401,453.451529,454.447331,465.595869,456.697513,491.791222,491.234880,485.256372,486.462712,488.853939,516.079530,502.212470,485.352711,464.695225,470.588628,468.813347,470.159250,469.693760,467.284235,409.015925,507.923342,504.781471,504.440841,500.831597,354.521242,364.260310,350.404305,355.651696,374.108621,348.492670,43.367801,527.218474,466.631070,462.775013,467.706124,468.259243,470.002455,468.179177,527.945912,478.232277,500.705114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8869495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,485.782741,477.721202,487.524073,492.640914,471.372116,494.759632,181.669115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8869496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2227.759711,2281.914310,2251.710160,2181.482982,2305.830288,2245.741296,3161.973990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8869497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [81]:
test

,id,date,country,store,product
0,230130,2017-01-01,Canada,Discount Stickers,Holographic Goose
1,230131,2017-01-01,Canada,Discount Stickers,Kaggle
2,230132,2017-01-01,Canada,Discount Stickers,Kaggle Tiers
3,230133,2017-01-01,Canada,Discount Stickers,Kerneler
4,230134,2017-01-01,Canada,Discount Stickers,Kerneler Dark Mode
...,...,...,...,...,...
98545,328675,2019-12-31,Singapore,Premium Sticker Mart,Holographic Goose
98546,328676,2019-12-31,Singapore,Premium Sticker Mart,Kaggle
98547,328677,2019-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers
98548,328678,2019-12-31,Singapore,Premium Sticker Mart,Kerneler


In [11]:
class Config:
    
    state = 4152
    n_splits = 5
    early_stop = 200
        
    target = 'num_sold'
    train = pd.read_csv('./datadown/train.csv', index_col='id')
    test = pd.read_csv('./datadown/test.csv', index_col='id')
    submission = pd.read_csv('./datadown/sample_submission.csv')

    original_data = 'N'
    outliers = 'N'
    log_trf = 'Y'
    scaler_trf = 'N'
    feature_eng = 'Y'
    missing = 'Y'

In [83]:
test.shape[0]

98550

In [119]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_absolute_percentage_error

class Ensemble:
    def __init__(self, n_splits=5, random_state=42):
        self.n_splits = n_splits
        self.state = random_state
    
    def ensemble(self, X, y, test):
        scores = [] 
        oof_pred = np.zeros(X.shape[0]) 
        test_pred = np.zeros(test.shape[0]) 
        feature_importances = np.zeros(X.shape[1])
        
        model = BayesianRidge() 
        kf = KFold(n_splits=self.n_splits, random_state=self.state, shuffle=True) 
 
        for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X, y)): 
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx] 
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx] 
 
            model.fit(X_train, y_train) 
 
            # Calculate feature importances using coefficients
            feature_importances += np.abs(model.coef_) / self.n_splits
 
            y_pred_probs = model.predict(X_val) 
            oof_pred[val_idx] = y_pred_probs 
            test_pred += model.predict(test) / self.n_splits 
 
            score = mean_absolute_percentage_error(y_val, y_pred_probs) 
            scores.append(score) 
             
        # Normalize feature importances
        feature_importances = feature_importances / np.sum(feature_importances)
        
        return {
            'mean_score': np.mean(scores), 
            'oof_pred': oof_pred, 
            'test_pred': test_pred,
            'feature_importances': feature_importances
        }

# Usage example
e = Ensemble()

submission_ = e.ensemble(
    oof_all.drop(columns=["num_sold_origin", "num_sold"]),
    oof_all[['num_sold_origin']],
    test_all.iloc[: test.shape[0], :],
)
feature_importances = submission_['feature_importances']

# To display feature importances
for name, importance in zip(
    oof_all.drop(columns=["num_sold_origin", "num_sold"]).columns, feature_importances
):
    print(f"{name}: {importance:.4f}")

CAT3_submissions_1: 0.0022
CAT_submissions_1: 0.0027
LGBM2_submissions_1: 0.0083
LGBM3_submissions_1: 0.0146
LGBM4_submissions_1: 0.0023
LGBM5_submissions_1: 0.0094
LGBM6_submissions_1: 0.0039
LGBM7_submissions_1: 0.0012
XGB2_submissions_1: 0.0071
XGB3_submissions_1: 0.0047
XGB4_submissions_1: 0.0061
XGB5_submissions_1: 0.0072
CAT3_submissions_2: 0.0013
CAT5_submissions_2: 0.0033
CAT6_submissions_2: 0.0012
CAT_submissions_2: 0.0046
LGBM2_submissions_2: 0.0081
LGBM3_submissions_2: 0.0211
LGBM4_submissions_2: 0.0051
LGBM5_submissions_2: 0.0055
LGBM6_submissions_2: 0.0092
LGBM7_submissions_2: 0.0017
NN_submissions_2: 0.0001
XGB2_submissions_2: 0.0165
XGB3_submissions_2: 0.0125
XGB4_submissions_2: 0.0086
XGB5_submissions_2: 0.0089
LGBM2_submissions_3: 0.0212
LGBM3_submissions_3: 0.0046
LGBM4_submissions_3: 0.0270
LGBM5_submissions_3: 0.0158
LGBM6_submissions_3: 0.0107
LGBM7_submissions_3: 0.0053
NN_submissions_3: 0.0000
BRidge_submissions_4: 0.3456
LGBM2_submissions_4: 0.0134
LGBM3_submiss

In [123]:
submission_['test_pred']

array([ 137.48170302, 1293.08291442,  946.68715454, ..., 3260.39914325,
       1538.85168705, 1743.98043291])

In [113]:
len(submission_[1])

230130

In [114]:
len(submission_[2])

98550

In [120]:
sb = pd.read_csv("./datadown/sample_submission.csv")

In [126]:
sb['num_sold'] = np.round(np.clip( submission_['test_pred'], 0, 6000))

In [127]:
sb

,id,num_sold
0,230130,137.0
1,230131,1293.0
2,230132,947.0
3,230133,514.0
4,230134,612.0
...,...,...
98545,328675,415.0
98546,328676,3475.0
98547,328677,3260.0
98548,328678,1539.0


In [128]:
sb.to_csv("./submission_0/submission.csv", index=None)
